# Лабораторная работа "Решение СЛАУ"

В этой лабораторной работе мы найдем решение системы линейных алгебраических уравнений.

Вспомогательный код для вычисления:

In [1]:
import math as m
import numpy as np
import random as r

def set_matrix(A, diag):
    for x in range(len(A)):
        for y in range(len(A[x])):
            A[x][y] = r.randint(-9, 9) + 10 * len(A) * (x == y) * diag
    return A

def gaussian(A, f):
    A = A.copy()
    f = f.copy()
    x = np.zeros((len(A), 1))

    for row in range(len(A) - 1):
        ind = np.argmax(A[row:, row]) + row
        A[row], A[ind] = A[ind].copy(), A[row].copy()
        f[row], f[ind] = f[ind].copy(), f[row].copy()

        for i in range(row + 1, len(A)):
            a = A[i][row]
            A[i] -= A[row] * a / A[row][row]
            f[i] -= f[row] * a / A[row][row]

    for row in range(len(A) - 1, -1, -1):
        sum = 0
        for i in range(row + 1, len(A)):
            sum -= x[i] * A[row][i]

        sum += f[row]
        x[row] = sum / A[row][row]

    return x

def jacobi(A, f, eps):
    x = np.zeros((len(A), 1))
    nx = np.ones((len(A), 1))
    iter_num = 0

    while True:
        x = nx.copy()

        for i in range(len(A)):
            sum = 0
            for j in range(len(A)):
                sum += A[i][j] * x[j] * (i != j)

            nx[i] = (f[i] - sum) / A[i][i]

        iter_num += 1
        if max(np.absolute(x - nx)) < eps:
            break

    return x, iter_num

def seidel(A, f, eps):
    x = np.zeros((len(A), 1))
    iter_num = 0

    while True:
        old_x = x.copy()

        for i in range(len(A)):
            sum = 0
            for j in range(len(A)):
                sum += A[i][j] * x[j] * (i != j)

            x[i] = (f[i] - sum) / A[i][i]

        iter_num += 1
        if max(np.absolute(x - old_x)) < eps:
            break

    return x, iter_num

def steepest_descent(A, f, eps):
    f = np.dot(A.transpose(), f)
    A = np.dot(A.transpose(), A)
    x = np.zeros((len(A), 1))
    iter_num = 0

    while True:
        old_x = x.copy()

        r = np.dot(A, x) - f
        tau = np.dot(r.transpose(), r) / np.dot(np.dot(A, r).transpose(), r)
        x = x - tau * r

        iter_num += 1
        if max(np.absolute(x - old_x)) < eps:
            break

    return x, iter_num

def min_discrepancy(A, f, eps):
    f = np.dot(A.transpose(), f)
    A = np.dot(A.transpose(), A)
    x = np.zeros((len(A), 1))
    iter_num = 0

    while True:
        old_x = x.copy()

        r = np.dot(A, x) - f
        Ar = np.dot(A, r)
        tau = np.dot(Ar.transpose(), r) / np.dot(Ar.transpose(), Ar)
        x = x - tau * r

        iter_num += 1
        if max(np.absolute(x - old_x)) < eps:
            break

    return x, iter_num

def average_iterations(method, iter_n, mat_n, eps):
    sum = 0

    for i in range(iter_n):
        A = set_matrix(np.zeros((mat_n, mat_n)), 1)
        f = set_matrix(np.zeros((mat_n, 1)), 0)

        sum += method(A, f, eps)[1]

    return sum / iter_n

## Решение СЛАУ

В качестве матрицы возьмем случайную с диагональным преобладанием чтобы работали все методы. \
Матрица размером $$100 \times 100$$
Для итерационных методов возьмем точность $$\epsilon=10^{-10}$$

In [2]:
n = 100
eps = 1.0e-10

A = set_matrix(np.zeros((n, n)), 1)
f = set_matrix(np.zeros((n, 1)), 0)

Для проверки правильности решений СЛАУ вычисленных разными методами подставим их в систему.

### Метод Гаусса

In [3]:
x = gaussian(A, f)
print(max(np.absolute(np.dot(A, x) - f)))

[1.42108547e-14]


### Метод Якоби

In [4]:
x = jacobi(A, f, eps)[0]
print(max(np.absolute(np.dot(A, x) - f)))

[6.40484576e-09]


### Метод Зейделя

In [5]:
x = seidel(A, f, eps)[0]
print(max(np.absolute(np.dot(A, x) - f)))

[2.03784545e-10]


### Метод наискорейшего спуска

In [6]:
x = steepest_descent(A, f, eps)[0]
print(max(np.absolute(np.dot(A, x) - f)))

[9.49333989e-09]


### Метод наименьших невязок

In [7]:
x = min_discrepancy(A, f, eps)[0]
print(max(np.absolute(np.dot(A, x) - f)))

[9.88704807e-09]


## Сравнение количества итераций

Сравним количество итераций для каждого метода.

In [8]:
iter_num = 50

print("Метод Якоби: ", average_iterations(jacobi, iter_num, n, eps))
print("Метод Зейделя: ", average_iterations(seidel, iter_num, n, eps))
print("Метод наискорейшего спуска: ", average_iterations(steepest_descent, iter_num, n, eps))
print("Метод наименьших невязок: ", average_iterations(min_discrepancy, iter_num, n, eps))

Метод Якоби:  10.0
Метод Зейделя:  7.0
Метод наискорейшего спуска:  10.36
Метод наименьших невязок:  10.4
